In [1]:
!/Users/taoguinot/miniconda3/bin/python -m pip install --upgrade pip
!/Users/taoguinot/miniconda3/bin/python -m pip install torch
!/Users/taoguinot/miniconda3/bin/python -m pip install torchvision

In [ ]:
!pip show torch


In [ ]:
!which python

In [2]:
import torch
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler
import argparse
import os
from torchvision import datasets, models, transforms
from network.classifier import *
from network.transform import mesonet_data_transforms

In [3]:
train_path="./data/train"
val_path ="./data/val"
batch_size = 64
epoches = 50
output_path = "./output/mesonet/best.pkl"
model_name = "Mesonet4.pkl"

In [4]:
train_dataset = torchvision.datasets.ImageFolder(train_path, transform=mesonet_data_transforms['train'])
val_dataset = torchvision.datasets.ImageFolder(val_path, transform=mesonet_data_transforms['val'])
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=8)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last=False, num_workers=8)
train_dataset_size = len(train_dataset)
val_dataset_size = len(val_dataset)

print(f"train size: {train_dataset_size}")
print(f"val size: {val_dataset_size}")

train size: 140002
val size: 39428


In [6]:
import torch
print(torch.cuda.is_available())  # Should return False on Mac ARM
print(torch.backends.mps.is_available())  # Check if Metal Performance Shaders (MPS) is supported
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)


False
True
mps


In [7]:
# model
model = Meso4()
model = model.to(device)
criterion = nn.CrossEntropyLoss() # classification problem
optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.99), eps=1e-8)
scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

In [8]:
best_model_weights = model.state_dict()
best_acc = 0.0
iter = 0

for epoch in range(epoches):
    print(f"Epoch {epoch+1}/{epoches}")
    model = model.train()
    train_loss = 0.0
    train_corrects = 0.0
    val_loss = 0.0
    val_corrects = 0.0
    for (image, labels) in train_loader:
        print(image.shape)
        iter_loss = 0.0
        iterr_corrects = 0.0
        image = image.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(image)
        _, preds = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        iter_loss = loss.data.item()
        train_loss += iter_loss
        iter_corrects = torch.sum(preds == labels.data).to(torch.float32)
        train_corrects += iter_corrects
        iter += 1
    epoch_loss = train_loss / train_dataset_size
    epoch_acc = train_corrects / train_dataset_size
    print(f"Epoch train loss: {epoch_loss}; Acc: {epoch_acc}\n\n")

    model.eval()
    with torch.no_grad():
        for (image, labels) in val_loader:
            image = image.to(device)
            labels = labels.to(device)
            outputs = model(image)
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            val_loss += loss.data.item()
            val_corrects += torch.sum(preds == labels.data).to(torch.float32)
        epoch_loss = val_loss / val_dataset_size
        epoch_acc = val_corrects / val_dataset_size
        print('epoch val loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = model.state_dict()
    scheduler.step()
    if not (epoch % 10):
    #Save the model trained with multiple gpu
    #torch.save(model.module.state_dict(), os.path.join(output_path, str(epoch) + '_' + model_name))
        torch.save(model.state_dict(), os.path.join(output_path, str(epoch) + '_' + model_name))
print('Best val Acc: {:.4f}'.format(best_acc))
model.load_state_dict(best_model_wts)
#torch.save(model.module.state_dict(), os.path.join(output_path, "best.pkl"))
torch.save(model.state_dict(), os.path.join(output_path, "best.pkl"))



Epoch 1/50
torch.Size([64, 3, 256, 256])


/Users/taoguinot/miniconda3/lib/python3.12/site-packages/torch/nn/functional.py:1538: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


AttributeError: 'float' object has no attribute 'dim'